In [2]:
import requests
import json
import pandas as pd
import io
pd.set_option('display.max_rows', 500)

# New York Times Dataset

## Historical Cases and Deaths by Counties

In [8]:
from io import StringIO
url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv'
s = requests.get(url).text
df6 = pd.read_csv(StringIO(s))
print(df6.shape)
print(df6.columns)
df6.head(100)

(891309, 6)
Index(['date', 'county', 'state', 'fips', 'cases', 'deaths'], dtype='object')


,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0
5,2020-01-25,Orange,California,6059.0,1,0.0
6,2020-01-25,Cook,Illinois,17031.0,1,0.0
7,2020-01-25,Snohomish,Washington,53061.0,1,0.0
8,2020-01-26,Maricopa,Arizona,4013.0,1,0.0
9,2020-01-26,Los Angeles,California,6037.0,1,0.0


## Mask Wearing Data

In [11]:
url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/mask-use/mask-use-by-county.csv'
s = requests.get(url).text
df9 = pd.read_csv(StringIO(s))
print(df9.shape)
print(df9.columns)
df9.head(100)

(3142, 6)
Index(['COUNTYFP', 'NEVER', 'RARELY', 'SOMETIMES', 'FREQUENTLY', 'ALWAYS'], dtype='object')


,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
0,1001,0.053,0.074,0.134,0.295,0.444
1,1003,0.083,0.059,0.098,0.323,0.436
2,1005,0.067,0.121,0.120,0.201,0.491
3,1007,0.020,0.034,0.096,0.278,0.572
4,1009,0.053,0.114,0.180,0.194,0.459
5,1011,0.031,0.040,0.144,0.286,0.500
6,1013,0.102,0.053,0.257,0.137,0.451
7,1015,0.152,0.108,0.130,0.167,0.442
8,1017,0.117,0.037,0.150,0.136,0.560
9,1019,0.135,0.027,0.161,0.158,0.520


# CDC Datatsets

## Provisional COVID-19 Death Counts by Sex, Age, and State

In [12]:
url = 'https://data.cdc.gov/resource/9bhg-hcku.json'
df10 = pd.read_json(url)
print(df10.shape)
print(df10.columns)
df10.head(100)

(1000, 13)
Index(['data_as_of', 'start_week', 'end_week', 'state', 'sex', 'age_group_new',
       'covid_19_deaths', 'total_deaths', 'pneumonia_deaths',
       'pneumonia_and_covid_19_deaths', 'influenza_deaths',
       'pneumonia_influenza_or_covid', 'footnote'],
      dtype='object')


,data_as_of,start_week,end_week,state,sex,age_group_new,covid_19_deaths,total_deaths,pneumonia_deaths,pneumonia_and_covid_19_deaths,influenza_deaths,pneumonia_influenza_or_covid,footnote
0,2020-12-30T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,United States,All Sexes,All Ages,301679.0,2902664.0,285361.0,138917.0,6946.0,453994.0,NaN
1,2020-12-30T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,United States,All Sexes,Under 1 year,32.0,16076.0,155.0,6.0,16.0,197.0,NaN
2,2020-12-30T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,United States,All Sexes,0-17 years,147.0,28488.0,443.0,27.0,125.0,688.0,NaN
3,2020-12-30T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,United States,All Sexes,1-4 years,19.0,2969.0,96.0,3.0,43.0,155.0,NaN
4,2020-12-30T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,United States,All Sexes,5-14 years,51.0,4810.0,131.0,9.0,53.0,226.0,NaN
5,2020-12-30T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,United States,All Sexes,15-24 years,483.0,30975.0,627.0,172.0,55.0,989.0,NaN
6,2020-12-30T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,United States,All Sexes,18-29 years,1180.0,54787.0,1456.0,490.0,103.0,2239.0,NaN
7,2020-12-30T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,United States,All Sexes,25-34 years,2087.0,63554.0,2278.0,931.0,156.0,3574.0,NaN
8,2020-12-30T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,United States,All Sexes,30-49 years,12341.0,190623.0,11526.0,5660.0,580.0,18703.0,NaN
9,2020-12-30T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,United States,All Sexes,35-44 years,5398.0,89922.0,5151.0,2429.0,253.0,8342.0,NaN


## United States COVID-19 Cases and Deaths by State over Time

In [13]:
url = 'https://data.cdc.gov/resource/9mfq-cb36.json'
df11 = pd.read_json(url)
print(df11.shape)
print(df11.columns)
df11.head(100)

(1000, 15)
Index(['submission_date', 'state', 'tot_cases', 'new_case', 'pnew_case',
       'tot_death', 'new_death', 'pnew_death', 'created_at', 'consent_cases',
       'consent_deaths', 'conf_cases', 'prob_cases', 'conf_death',
       'prob_death'],
      dtype='object')


,submission_date,state,tot_cases,new_case,pnew_case,tot_death,new_death,pnew_death,created_at,consent_cases,consent_deaths,conf_cases,prob_cases,conf_death,prob_death
0,2020-12-08T00:00:00.000,NM,109947,0,0.0,1756,0,0.0,2020-12-09 14:45:40.234,N/A,Not agree,NaN,NaN,NaN,NaN
1,2021-01-01T00:00:00.000,FL,1300528,0,6063.0,21673,0,7.0,2021-01-02 14:50:51.219,Not agree,Not agree,NaN,NaN,NaN,NaN
2,2020-04-30T00:00:00.000,IA,7145,302,0.0,162,14,0.0,2020-05-01 21:00:19.025,Not agree,Not agree,NaN,NaN,NaN,NaN
3,2020-06-25T00:00:00.000,NE,18346,125,0.0,260,3,0.0,2020-06-26 19:18:27.809,Not agree,Not agree,NaN,NaN,NaN,NaN
4,2020-02-24T00:00:00.000,CA,10,0,NaN,0,0,NaN,2020-03-26 16:22:39.452,Not agree,Not agree,NaN,NaN,NaN,NaN
5,2020-11-14T00:00:00.000,VA,201961,1161,191.0,3800,1,0.0,2020-11-15 15:12:13.050,Agree,Agree,183455.0,18506.0,3527.0,273.0
6,2020-12-02T00:00:00.000,NY,346492,5775,0.0,10117,56,0.0,2020-12-03 15:21:08.669,Not agree,Not agree,NaN,NaN,NaN,NaN
7,2020-01-23T00:00:00.000,WY,0,0,NaN,0,0,NaN,2020-03-26 16:22:39.452,Agree,Agree,NaN,NaN,NaN,NaN
8,2020-05-05T00:00:00.000,GA,28196,602,1.0,1258,47,0.0,2020-05-06 12:23:08.000,Agree,Agree,28182.0,14.0,1258.0,0.0
9,2020-01-29T00:00:00.000,MA,0,0,NaN,0,0,NaN,2020-03-26 16:22:39.452,Agree,Agree,NaN,NaN,NaN,NaN


## Conditions Contributing to Deaths Involving COVID-19 by Age Group and State

In [14]:
url = 'https://data.cdc.gov/resource/hk9y-quqm.json'
df12 = pd.read_json(url)
print(df12.shape)
print(df12.columns)
df12.head(100)

(1000, 11)
Index(['data_as_of', 'start_week', 'end_week', 'state', 'condition_group',
       'condition', 'icd10_codes', 'age_group', 'covid_19_deaths',
       'number_of_mentions', 'flag'],
      dtype='object')


,data_as_of,start_week,end_week,state,condition_group,condition,icd10_codes,age_group,covid_19_deaths,number_of_mentions,flag
0,2020-12-27T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,US,Respiratory diseases,Influenza and pneumonia,J09-J18,0-24,186.0,193.0,NaN
1,2020-12-27T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,US,Respiratory diseases,Influenza and pneumonia,J09-J18,25-34,924.0,941.0,NaN
2,2020-12-27T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,US,Respiratory diseases,Influenza and pneumonia,J09-J18,35-44,2387.0,2431.0,NaN
3,2020-12-27T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,US,Respiratory diseases,Influenza and pneumonia,J09-J18,45-54,6791.0,6964.0,NaN
4,2020-12-27T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,US,Respiratory diseases,Influenza and pneumonia,J09-J18,55-64,16901.0,17298.0,NaN
5,2020-12-27T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,US,Respiratory diseases,Influenza and pneumonia,J09-J18,65-74,29328.0,29935.0,NaN
6,2020-12-27T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,US,Respiratory diseases,Influenza and pneumonia,J09-J18,75-84,35254.0,35791.0,NaN
7,2020-12-27T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,US,Respiratory diseases,Influenza and pneumonia,J09-J18,85+,34999.0,35304.0,NaN
8,2020-12-27T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,US,Respiratory diseases,Influenza and pneumonia,J09-J18,Not stated,3.0,3.0,NaN
9,2020-12-27T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,US,Respiratory diseases,Influenza and pneumonia,J09-J18,All Ages,126773.0,128860.0,NaN


## COVID-19 Case Surveillance Public Use Data

In [15]:
url = 'https://data.cdc.gov/resource/vbim-akqf.json'
df13 = pd.read_json(url)
print(df13.shape)
print(df13.columns)
df13.head(100)

(1000, 12)
Index(['cdc_case_earliest_dt', 'cdc_report_dt', 'pos_spec_dt',
       'current_status', 'sex', 'age_group', 'race_ethnicity_combined',
       'hosp_yn', 'icu_yn', 'death_yn', 'medcond_yn', 'onset_dt'],
      dtype='object')


,cdc_case_earliest_dt,cdc_report_dt,pos_spec_dt,current_status,sex,age_group,race_ethnicity_combined,hosp_yn,icu_yn,death_yn,medcond_yn,onset_dt
0,2020-03-23T00:00:00.000,2020-03-31T00:00:00.000,2020-03-23T00:00:00.000,Laboratory-confirmed case,Female,0 - 9 Years,"Black, Non-Hispanic",Unknown,Unknown,Unknown,Unknown,NaN
1,2020-03-22T00:00:00.000,2020-03-23T00:00:00.000,2020-03-23T00:00:00.000,Laboratory-confirmed case,Female,0 - 9 Years,Hispanic/Latino,Yes,Unknown,Unknown,Unknown,NaN
2,2020-03-22T00:00:00.000,2020-03-22T00:00:00.000,2020-03-23T00:00:00.000,Laboratory-confirmed case,Female,0 - 9 Years,Hispanic/Latino,No,No,No,No,2020-03-22T00:00:00.000
3,2020-03-23T00:00:00.000,2020-03-23T00:00:00.000,2020-03-23T00:00:00.000,Laboratory-confirmed case,Female,0 - 9 Years,Hispanic/Latino,No,Missing,No,No,2020-03-23T00:00:00.000
4,2020-03-23T00:00:00.000,2020-03-23T00:00:00.000,2020-03-23T00:00:00.000,Laboratory-confirmed case,Female,0 - 9 Years,Hispanic/Latino,Unknown,Unknown,Unknown,Unknown,NaN
5,2020-03-23T00:00:00.000,2020-03-23T00:00:00.000,2020-03-23T00:00:00.000,Laboratory-confirmed case,Female,0 - 9 Years,Hispanic/Latino,No,Missing,No,No,2020-03-23T00:00:00.000
6,2020-03-23T00:00:00.000,2020-03-24T00:00:00.000,2020-03-23T00:00:00.000,Laboratory-confirmed case,Female,0 - 9 Years,Hispanic/Latino,Yes,Unknown,Unknown,Unknown,NaN
7,2020-03-23T00:00:00.000,2020-03-23T00:00:00.000,2020-03-23T00:00:00.000,Laboratory-confirmed case,Male,0 - 9 Years,Missing,Missing,Missing,Missing,Missing,NaN
8,2020-03-16T00:00:00.000,2020-03-31T00:00:00.000,2020-03-23T00:00:00.000,Laboratory-confirmed case,Male,0 - 9 Years,Missing,Yes,No,No,Yes,2020-03-16T00:00:00.000
9,2020-03-22T00:00:00.000,2020-03-25T00:00:00.000,2020-03-23T00:00:00.000,Laboratory-confirmed case,Male,0 - 9 Years,"Black, Non-Hispanic",Yes,Unknown,Unknown,Unknown,NaN


## Provisional COVID-19 Death Counts in the US by County

In [16]:
url = 'https://data.cdc.gov/resource/kn79-hsxy.json'
df14 = pd.read_json(url)
print(df14.shape)
print(df14.columns)
df14.head(100)

(1000, 9)
Index(['data_as_of', 'start_week', 'end_week', 'state_name', 'county_name',
       'county_fips_code', 'urban_rural_code', 'covid_death', 'total_death'],
      dtype='object')


,data_as_of,start_week,end_week,state_name,county_name,county_fips_code,urban_rural_code,covid_death,total_death
0,2020-12-30T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,AK,Anchorage Borough,2020,Medium metro,95,1913
1,2020-12-30T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,AK,Fairbanks North Star Borough,2090,Small metro,20,471
2,2020-12-30T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,AL,Autauga County,1001,Medium metro,39,458
3,2020-12-30T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,AL,Baldwin County,1003,Small metro,109,2040
4,2020-12-30T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,AL,Barbour County,1005,Noncore,17,253
5,2020-12-30T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,AL,Bibb County,1007,Large fringe metro,20,175
6,2020-12-30T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,AL,Blount County,1009,Large fringe metro,17,449
7,2020-12-30T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,AL,Butler County,1013,Noncore,27,208
8,2020-12-30T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,AL,Calhoun County,1015,Small metro,145,1453
9,2020-12-30T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,AL,Chambers County,1017,Micropolitan,11,340


## Excess Deaths Associated with COVID-19

In [17]:
url = 'https://data.cdc.gov/resource/xkkf-xrst.json'
df15 = pd.read_json(url)
print(df15.shape)
print(df15.columns)
df15.head(100)

(1000, 17)
Index(['week_ending_date', 'state', 'observed_number', 'upper_bound_threshold',
       'exceeds_threshold', 'average_expected_count', 'excess_lower_estimate',
       'excess_higher_estimate', 'year', 'total_excess_lower_estimate_in_2020',
       'total_excess_higher_estimate_in_2020', 'percent_excess_lower_estimate',
       'percent_excess_higher_estimate', 'type', 'outcome', 'note',
       'suppress'],
      dtype='object')


,week_ending_date,state,observed_number,upper_bound_threshold,exceeds_threshold,average_expected_count,excess_lower_estimate,excess_higher_estimate,year,total_excess_lower_estimate_in_2020,total_excess_higher_estimate_in_2020,percent_excess_lower_estimate,percent_excess_higher_estimate,type,outcome,note,suppress
0,2017-01-14,Alabama,1130.0,1189,False,1028,0,102,2017,4952,7714,0.0,0.099339,Predicted (weighted),All causes,NaN,NaN
1,2017-01-21,Alabama,1048.0,1204,False,1042,0,6,2017,4952,7714,0.0,0.006182,Predicted (weighted),All causes,NaN,NaN
2,2017-01-28,Alabama,1026.0,1222,False,1060,0,0,2017,4952,7714,0.0,0.000000,Predicted (weighted),All causes,NaN,NaN
3,2017-02-04,Alabama,1036.0,1218,False,1056,0,0,2017,4952,7714,0.0,0.000000,Predicted (weighted),All causes,NaN,NaN
4,2017-02-11,Alabama,1058.0,1207,False,1049,0,9,2017,4952,7714,0.0,0.008618,Predicted (weighted),All causes,NaN,NaN
5,2017-02-18,Alabama,1060.0,1215,False,1054,0,6,2017,4952,7714,0.0,0.005907,Predicted (weighted),All causes,NaN,NaN
6,2017-02-25,Alabama,1099.0,1205,False,1045,0,54,2017,4952,7714,0.0,0.051322,Predicted (weighted),All causes,NaN,NaN
7,2017-03-04,Alabama,1081.0,1194,False,1034,0,47,2017,4952,7714,0.0,0.045004,Predicted (weighted),All causes,NaN,NaN
8,2017-03-11,Alabama,1011.0,1183,False,1026,0,0,2017,4952,7714,0.0,0.000000,Predicted (weighted),All causes,NaN,NaN
9,2017-03-18,Alabama,1067.0,1173,False,1015,0,52,2017,4952,7714,0.0,0.051366,Predicted (weighted),All causes,NaN,NaN


## Provisional COVID-19 Death Counts by Sex, Age, and Week

In [18]:
url = 'https://data.cdc.gov/resource/vsak-wrfu.json'
df16 = pd.read_json(url)
print(df16.shape)
print(df16.columns)
df16.head(100)

(1000, 8)
Index(['data_as_of', 'state', 'mmwr_week', 'week_ending_date', 'sex',
       'age_group', 'total_deaths', 'covid_19_deaths'],
      dtype='object')


,data_as_of,state,mmwr_week,week_ending_date,sex,age_group,total_deaths,covid_19_deaths
0,2020-12-30T00:00:00.000,United States,5,2020-02-01T00:00:00.000,All Sex,All Ages,58795,0
1,2020-12-30T00:00:00.000,United States,5,2020-02-01T00:00:00.000,All Sex,Under 1 year,390,0
2,2020-12-30T00:00:00.000,United States,5,2020-02-01T00:00:00.000,All Sex,1-4 years,65,0
3,2020-12-30T00:00:00.000,United States,5,2020-02-01T00:00:00.000,All Sex,5-14 years,111,0
4,2020-12-30T00:00:00.000,United States,5,2020-02-01T00:00:00.000,All Sex,15-24 years,574,0
5,2020-12-30T00:00:00.000,United States,5,2020-02-01T00:00:00.000,All Sex,25-34 years,1134,0
6,2020-12-30T00:00:00.000,United States,5,2020-02-01T00:00:00.000,All Sex,35-44 years,1666,0
7,2020-12-30T00:00:00.000,United States,5,2020-02-01T00:00:00.000,All Sex,45-54 years,3337,0
8,2020-12-30T00:00:00.000,United States,5,2020-02-01T00:00:00.000,All Sex,55-64 years,7483,0
9,2020-12-30T00:00:00.000,United States,5,2020-02-01T00:00:00.000,All Sex,65-74 years,11666,0


## Provisional Death Counts for COVID-19: Distribution of Deaths by Race and Hispanic Origin

In [19]:
url = 'https://data.cdc.gov/resource/pj7m-y5uh.json'
df17 = pd.read_json(url)
print(df17.shape)
print(df17.columns)
df17.head(100)

(208, 13)
Index(['data_as_of', 'start_week', 'end_week', 'state', 'indicator',
       'non_hispanic_white', 'non_hispanic_black_african_american',
       'non_hispanic_american_indian_alaska_native',
       'non_hispanic_asian_pacific_islander', 'nh_nhopi',
       'hispanic_latino_total', 'other', 'footnote'],
      dtype='object')


,data_as_of,start_week,end_week,state,indicator,non_hispanic_white,non_hispanic_black_african_american,non_hispanic_american_indian_alaska_native,non_hispanic_asian_pacific_islander,nh_nhopi,hispanic_latino_total,other,footnote
0,2020-12-30T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,United States,Count of COVID-19 deaths,177636.0,51054.0,3318.0,10725.0,535.0,56097.0,2314.0,NaN
1,2020-12-30T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,United States,Distribution of COVID-19 deaths (%),58.9,16.9,1.1,3.6,0.2,18.6,0.8,NaN
2,2020-12-30T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,United States,Unweighted distribution of population (%),60.1,12.5,0.7,5.8,0.2,18.5,2.2,NaN
3,2020-12-30T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,United States,Weighted distribution of population (%),41.2,14.9,0.4,9.5,0.1,31.9,2.0,NaN
4,2020-12-30T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,Alabama,Count of COVID-19 deaths,3431.0,1655.0,NaN,26.0,NaN,117.0,NaN,One or more data cells have counts between 1-9...
5,2020-12-30T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,Alabama,Distribution of COVID-19 deaths (%),65.4,31.5,NaN,0.5,NaN,2.2,NaN,One or more data cells have counts between 1-9...
6,2020-12-30T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,Alabama,Unweighted distribution of population (%),65.3,26.5,0.6,1.5,0.1,4.6,1.6,NaN
7,2020-12-30T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,Alabama,Weighted distribution of population (%),55.6,36.5,0.4,1.9,0.0,4.1,1.5,NaN
8,2020-12-30T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,Alaska,Count of COVID-19 deaths,50.0,NaN,55.0,18.0,11.0,NaN,NaN,One or more data cells have counts between 1-9...
9,2020-12-30T00:00:00.000,2020-02-01T00:00:00.000,2020-12-26T00:00:00.000,Alaska,Distribution of COVID-19 deaths (%),35.0,NaN,38.5,12.6,7.7,NaN,NaN,One or more data cells have counts between 1-9...


## COVID-19 Vaccine Distribution Allocastions by Jurisdiction - Pfizer

In [20]:
url = 'https://data.cdc.gov/resource/saz5-9hgg.json'
df18 = pd.read_json(url)
print(df18.shape)
print(df18.columns)
df18.head(100)

(66, 12)
Index(['jurisdiction', 'hhs_region', 'first_doses_12_14',
       'second_doses_shipment_12_14', 'doses_allocated_week_of_12_21',
       'second_dose_shipment_12_17', 'doses_allocated_week_of_12_28',
       'second_dose_shipment_12_28', 'doses_allocated_week_of_01_04',
       'second_dose_shipment_01_04',
       'total_pfizer_allocation_first_dose_shipments',
       'total_allocation_pfizer_second_dose_shipments'],
      dtype='object')


,jurisdiction,hhs_region,first_doses_12_14,second_doses_shipment_12_14,doses_allocated_week_of_12_21,second_dose_shipment_12_17,doses_allocated_week_of_12_28,second_dose_shipment_12_28,doses_allocated_week_of_01_04,second_dose_shipment_01_04,total_pfizer_allocation_first_dose_shipments,total_allocation_pfizer_second_dose_shipments
0,Connecticut,Region 1,"31,200","31,200","22,425","22,425","28,275","28,275","22,425","22,425","104,325","104,325"
1,Maine,Region 1,"12,675","12,675","8,775","8,775","10,725","10,725","8,775","8,775","40,950","40,950"
2,Massachusetts,Region 1,"59,475","59,475","42,900","42,900","54,600","54,600","42,900","42,900","199,875","199,875"
3,New Hampshire,Region 1,"12,675","12,675","8,775","8,775","10,725","10,725","8,775","8,775","40,950","40,950"
4,Rhode Island,Region 1,"9,750","9,750","6,825","6,825","8,775","8,775","6,825","6,825","32,175","32,175"
5,Vermont,Region 1,"5,850","5,850","4,875","4,875","5,850","5,850","3,900","3,900","20,475","20,475"
6,New Jersey,Region 2,"76,050","76,050","53,625","53,625","68,250","68,250","53,625","53,625","251,550","251,550"
7,New York,Region 2,"96,525","96,525","68,250","68,250","87,750","87,750","68,250","68,250","320,775","320,775"
8,New York City,Region 2,"73,125","73,125","51,675","51,675","66,300","66,300","51,675","51,675","242,775","242,775"
9,Puerto Rico,Region 2,"30,225","30,225","21,450","21,450","27,300","27,300","21,450","21,450","100,425","100,425"


## COVID-19 Vaccine Distribution Allocations by Jurisdiction - Moderna

In [21]:
url = 'https://data.cdc.gov/resource/b7pe-5nws.json'
df19 = pd.read_json(url)
print(df19.shape)
print(df19.columns)
df19.head(100)

(66, 10)
Index(['jurisdiction', 'hhs_region', 'doses_allocated_week_12_21',
       'second_dose_shipment_12_21', 'doses_allocated_week_12_28',
       'second_dose_shipment_12_28', 'doses_allocated_week_01_04',
       'second_dose_shipment_01_04',
       'total_moderna_allocation_first_dose_shipments',
       'total_allocation_moderna_second_dose_shipments'],
      dtype='object')


,jurisdiction,hhs_region,doses_allocated_week_12_21,second_dose_shipment_12_21,doses_allocated_week_12_28,second_dose_shipment_12_28,doses_allocated_week_01_04,second_dose_shipment_01_04,total_moderna_allocation_first_dose_shipments,total_allocation_moderna_second_dose_shipments
0,Connecticut,Region 1,"63,300","63,300","21,900","21,900","21,800","21,800","107,000","107,000"
1,Maine,Region 1,"24,200","24,200","8,400","8,400","8,300","8,300","40,900","40,900"
2,Massachusetts,Region 1,"121,900","121,900","42,100","42,100","42,000","42,000","206,000","206,000"
3,New Hampshire,Region 1,"24,200","24,200","8,400","8,400","8,400","8,400","41,000","41,000"
4,Rhode Island,Region 1,"19,000","19,000","6,600","6,600","6,600","6,600","32,200","32,200"
5,Vermont,Region 1,"11,400","11,400","4,000","4,000","3,900","3,900","19,300","19,300"
6,New Jersey,Region 2,"154,500","154,500","53,400","53,400","53,200","53,200","261,100","261,100"
7,New York,Region 2,"196,800","196,800","68,000","68,000","67,700","67,700","332,500","332,500"
8,New York City,Region 2,"149,400","149,400","51,600","51,600","51,400","51,400","252,400","252,400"
9,Puerto Rico,Region 2,"60,400","60,400","20,900","20,900","20,800","20,800","102,100","102,100"


# OPENICPSR State Policy Database 

In [36]:
xl = pd.ExcelFile('data/state-policy-database.xlsx', engine='openpyxl')
xl.sheet_names

['Variables', 'State policy changes ', 'Details', 'Contributors', 'Notes']

In [38]:
df20 = pd.read_excel('data/state-policy-database.xlsx', sheet_name='State policy changes ', engine='openpyxl')
print(df20.shape)
print(df20.columns)
df20.head(50)

(1004, 200)
Index(['STATE', 'POSTCODE', 'FIPS', 'STEMERG', 'CLSCHOOL', 'CLDAYCR',
       'OPNCLDCR', 'CLNURSHM', 'STAYHOME', 'STAYHOMENOGP',
       ...
       'MINWAGE2020', 'ALTMINWAGE2020', 'TIPPEDMINWAGE2020',
       'SMALLBIZMINWAGE2020', 'PLANMINWAGE2021', 'PLANMINWAGE2022',
       'PLANMINWAGE2023', 'PLANMINWAGE2024', 'PLANMINWAGE2025',
       'PLANMINWAGE2026'],
      dtype='object', length=200)


,STATE,POSTCODE,FIPS,STEMERG,CLSCHOOL,CLDAYCR,OPNCLDCR,CLNURSHM,STAYHOME,STAYHOMENOGP,...,MINWAGE2020,ALTMINWAGE2020,TIPPEDMINWAGE2020,SMALLBIZMINWAGE2020,PLANMINWAGE2021,PLANMINWAGE2022,PLANMINWAGE2023,PLANMINWAGE2024,PLANMINWAGE2025,PLANMINWAGE2026
0,State,State Abbreviation,FIPS Code,State of emergency,Date closed K-12 public schools,Closed day cares,Reopen day cares,Date banned visitors to nursing homes,Stay at home/ shelter in place,Stay at home order' issued but did not specifi...,...,2020 Minimum Wage,2020 Alternative Minimum Wage,2020 Minimum Wage for Tipped Workers,Different Minimum Wage for Smaller Businesses,[Planned] 2021 Minimum Wage,[Planned] 2022 Minimum Wage,[Planned] 2023 Minimum Wage,[Planned] 2024 Minimum Wage,[Planned] 2025 Minimum Wage,[Planned] 2026 Minimum Wage
1,category,NaN,NaN,state_of_emergency,physical_distance_closure,physical_distance_closure,Reopening,physical_distance_closure,shelter,shelter,...,minimum_wage,minimum_wage,minimum_wage,minimum_wage,minimum_wage,minimum_wage,minimum_wage,minimum_wage,minimum_wage,minimum_wage
2,type,note,note,start,start,start,end,start,start,start,...,quantity,quantity,quantity,attribute,quantity,quantity,quantity,quantity,quantity,quantity
3,unit,text,attribute,date,date,date,date,date,date,date,...,dollars,dollars,dollars,flag,dollars,dollars,dollars,dollars,dollars,dollars
4,Alabama,AL,1,2020-03-13 00:00:00,2020-03-20 00:00:00,2020-03-20 00:00:00,2020-05-23 00:00:00,2020-03-19 00:00:00,2020-04-04 00:00:00,0,...,.,.,2.13,0,.,.,.,.,.,.
5,Alaska,AK,2,2020-03-11 00:00:00,2020-03-16 00:00:00,0,0,0,2020-03-28 00:00:00,0,...,10.19,.,10.19,0,10.34,.,.,.,.,.
6,Arizona,AZ,4,2020-03-11 00:00:00,2020-03-16 00:00:00,0,0,0,2020-03-31 00:00:00,0,...,12,.,9,0,12.15,.,.,.,.,.
7,Arkansas,AR,5,2020-03-11 00:00:00,2020-03-17 00:00:00,0,0,2020-03-13 00:00:00,0,0,...,10,.,2.63,1,11,.,.,.,.,.
8,California,CA,6,2020-03-04 00:00:00,2020-03-23 00:00:00,0,0,0,2020-03-19 00:00:00,0,...,13,.,12,1,13,14,15,.,.,.
9,Colorado,CO,8,2020-03-11 00:00:00,2020-03-23 00:00:00,0,0,2020-03-12 00:00:00,2020-03-26 00:00:00,0,...,12,.,8.98,1,12.32,.,.,.,.,.


In [39]:
import covidcast
from datetime import date
data = covidcast.signal("fb-survey", "raw_wcli", date(2020, 9, 8), date(2021, 1, 1), "county")

ModuleNotFoundError: No module named 'covidcast'

In [41]:
import geopandas as gpd
data = gpd.read_file('https://opendata.arcgis.com/datasets/4cb598ae041348fb92270f102a6783cb_0.geojson')
print(data.shape)
print()

,OBJECTID,Countyname,ST_Name,ST_Abbr,ST_ID,FIPS,FatalityRa,Confirmedb,DeathsbyPo,PCTPOVALL_,...,Day_11,Day_12,Day_13,Day_14,NewCasebyP,Inpat_Occ,ICU_Occ,Shape__Area,Shape__Length,geometry
0,1,Autauga,Alabama,AL,01,01001,1.161440,7742.67,89.926440,13.8,...,53.0,48.0,61.0,39.0,66.545566,96.320346,100.000000,2.209382e+09,246839.865479,"POLYGON ((-86.41312 32.70739, -86.41219 32.526..."
1,2,Baldwin,Alabama,AL,01,01003,1.201650,6450.73,77.515113,9.8,...,200.0,166.0,203.0,125.0,49.994955,74.029401,120.089286,5.770469e+09,728445.072448,"MULTIPOLYGON (((-87.78878 31.29877, -87.78849 ..."
2,3,Barbour,Alabama,AL,01,01005,2.156863,6149.27,132.631325,30.9,...,7.0,20.0,27.0,0.0,8.038262,56.711409,88.571429,3.258643e+09,307285.154510,"POLYGON ((-85.25609 32.13767, -85.25569 32.135..."
3,4,Bibb,Alabama,AL,01,01007,2.444208,8401.79,205.357143,21.8,...,28.0,23.0,32.0,15.0,84.821429,19.047619,NaN,2.310715e+09,227886.963840,"POLYGON ((-87.02685 33.24646, -87.02572 33.209..."
4,5,Blount,Alabama,AL,01,01009,1.327434,8205.39,108.921162,13.2,...,38.0,54.0,79.0,16.0,29.391425,76.190476,92.857143,2.456058e+09,286306.840721,"POLYGON ((-86.44507 34.24954, -86.40902 34.205..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3326,3327,,,,0,00000,0.000000,0.00,0.000000,0.0,...,NaN,NaN,NaN,NaN,0.000000,27.740864,11.111111,NaN,NaN,None
3327,3328,,,,0,00000,0.000000,0.00,0.000000,0.0,...,NaN,NaN,NaN,NaN,0.000000,27.740864,11.111111,NaN,NaN,None
3328,3329,,,,0,00000,0.000000,0.00,0.000000,0.0,...,NaN,NaN,NaN,NaN,0.000000,27.740864,11.111111,NaN,NaN,None
3329,3330,,,,0,00000,0.000000,0.00,0.000000,0.0,...,NaN,NaN,NaN,NaN,0.000000,27.740864,11.111111,NaN,NaN,None


In [42]:
data.columns

Index(['OBJECTID', 'Countyname', 'ST_Name', 'ST_Abbr', 'ST_ID', 'FIPS',
       'FatalityRa', 'Confirmedb', 'DeathsbyPo', 'PCTPOVALL_', 'Unemployme',
       'Med_HH_Inc', 'State_Fata', 'DateChecke', 'EM_type', 'EM_date',
       'EM_notes', 'url', 'Thumbnail', 'Confirmed', 'Deaths', 'Age_85',
       'Age_80_84', 'Age_75_79', 'Age_70_74', 'Age_65_69', 'Beds_Licen',
       'Beds_Staff', 'Beds_ICU', 'Ventilator', 'POP_ESTIMA', 'POVALL_201',
       'Unemployed', 'Median_Hou', 'Recovered', 'Active', 'State_Conf',
       'State_Deat', 'State_Reco', 'State_Test', 'AgedPop', 'NewCases',
       'NewDeaths', 'TotalPop', 'NonHispWhP', 'BlackPop', 'AmIndop',
       'AsianPop', 'PacIslPop', 'OtherPop', 'TwoMorPop', 'HispPop', 'Wh_Alone',
       'Bk_Alone', 'AI_Alone', 'As_Alone', 'NH_Alone', 'SO_Alone', 'Two_More',
       'Not_Hisp', 'Age_Less15', 'Age_15_24', 'Age_25_34', 'Age_Over75',
       'Agetotal', 'NonHisp', 'Age_35_64', 'Age_65_74', 'Day_1', 'Day_2',
       'Day_3', 'Day_4', 'Day_5', 'Day_6'